In [1]:
"""
MODIFICADO DE pyredatam.py en https://github.com/abenassi/pyredatam/blob/master/pyredatam/pyredatam.py
"""

from __future__ import unicode_literals
from __future__ import print_function
from __future__ import with_statement

# funcion para escribir consulta redatam en lenguaje redatam
def frequency_query(area_break, variables, area_filter=None,
                   universe_filter=None, title=None):
    # RUNDEF section
    lines = _build_rundef_section(universe_filter)

    # TABLE section
    lines.append("TABLE TABLE1")
    lines.extend(_build_title(title))
    lines.append("    AS FREQUENCY")
    lines.append(_build_area_break(area_break))
    lines.append(_build_of_variables(variables))

    return "\n".join(lines)

# funcion para escribir parametros de la primera linea de la consulta
def _build_rundef_section(universe_filter):
    lines = ["RUNDEF Job"]
#     lines.extend(_build_area_filter(area_filter))
    lines.extend(_build_universe_filter(universe_filter))
    lines.append("")
    return lines

# funcion para escribir parametros de filtro universo
def _build_universe_filter(universe_filter):
    return ["    UNIVERSE " + universe_filter] if universe_filter else []

# funcion para escribir parametros de nombre de la tabla
def _build_title(title):
    return ['    TITLE "' + title + '"'] if title else []

# funcion para escribir parametros de nivel de salida de la consulta (areabreak)
def _build_area_break(area_break):
    if type(area_break) != list:
        area_break = [area_break]
    return "    AREABREAK {}".format(", ".join(area_break)) if area_break else []

# funcion para escribir parametros de variables de la consulta
def _build_of_variables(variables):
    if type(variables) != list:
        variables = [variables]
    return "    OF {}".format(", ".join(variables)) if variables else []

# ejemplo texto consulta 
# query = frequency_query("Departamento",["Vivienda.C2P2","Vivienda.C2P3"])
query = frequency_query("Departam",["Vivienda.VAREA","Poblacion.VASEXO"])

print(query)

RUNDEF Job

TABLE TABLE1
    AS FREQUENCY
    AREABREAK Departam
    OF Vivienda.VAREA, Poblacion.VASEXO


#### instalado
driver= google, firefox (driver) - .exe
selenium (package)

In [19]:
"""
MODIFICADO DE cpv2010arg.py en https://github.com/abenassi/pyredatam/blob/1480c481feb0698d54b59c3c17e52661a8c793df/pyredatam/cpv2010arg.py
"""
from __future__ import unicode_literals
from __future__ import print_function
from __future__ import with_statement

import pandas as pd
import requests
import os

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

#from pyvirtualdisplay import Display ### viene cn con abenassi
#from collections import OrderedDict ### viene cn con abenassi

from bs4 import BeautifulSoup

import json
import time

# espera hasta que aparezca texto / funcion-herramienta del scraping
def _wait_until_text(driver, tag_name, text):
    WebDriverWait(driver, 20).until(EC.text_to_be_present_in_element(
        (By.TAG_NAME, tag_name), text))

BASE_URL = "https://censos2017.inei.gob.pe/bininei/RpWebStats.exe/CmdSet?BASE=CPV2017DI&ITEM=PROGRED&lang=esp"
DRIVER_PATH = "C:\\Users\\Dieguinchi\\Desktop\\chromedriver102.exe"

# hace consulta "query" a redatam a traves de procesador estadistico online
def make_query(query, url=BASE_URL):
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)# el driver debe estar en archivos del pc. puede descargarse de https://chromedriver.chromium.org/downloads
    driver.get(url) # abre pagina web de redatam
    query_input = driver.find_element_by_tag_name("textarea")# abre pagina web de redatam
#     query_input.send_keys(query.decode("utf-8", "ignore"))
    query_input.send_keys(query) # escribe consulta en formulario
    submit = driver.find_element_by_name("Submit")
    submit.click()# ejecuta consulta
    _wait_until_text(driver,"p","Descargar en formato Excel")# espera que carguen los datos
    html = driver.page_source 
    driver.close() # cierra navegador
    return html # devuelve html con datos

#->>>> 

def download_excel():
    

# convierte datos de consulta en filas y columnas
def make_dataframe(html):
    bs = BeautifulSoup(html, "html5lib") # lee html con datos
    tables = bs.find_all("table") # busca tablas en los datos
    table = tables[0] # selecciona la primera tabla (la que contien los datos)
    rows = table.find_all("tr") # busca filas
    area_code = "" # resetea identificador de area (se usara mas adelante)
    df = pd.DataFrame(columns=["A","B","C","D","E","F"]) # define columnas
    for row in rows[5:-1]: # ordena las filas
        text_values = [td.get_text() for td in row.find_all("td")] # almacena texto de las filas en variable
        if text_values[1].find("AREA") != -1: # si contiene identificador de area, lo guarda
            area_code = text_values[1]
        values = text_values[1:] # descarta valosres vacios (columnas vacias)
        values.insert(0, area_code) # anade a cada fila su identificador de area
        df.loc[len(df)] = values # anade fila a base de datos
    return df # devuelve base de datos

make_dataframe(make_query(query)).head(15)


,A,B,C,D,E,F
0,AREA # 01,AREA # 01,Departamento: Amazonas,,,
1,AREA # 01,,,,,
2,AREA # 01,V: Área concepto censal,Casos,%,Acumulado %,
3,AREA # 01,Urbano censal,66 273,"47,57%","47,57%",
4,AREA # 01,Rural censal,73 055,"52,43%","100,00%",
5,AREA # 01,Total,139 328,"100,00%","100,00%",
6,AREA # 01,,,,,
7,AREA # 02,AREA # 02,Departamento: Áncash,,,
8,AREA # 02,,,,,
9,AREA # 02,V: Área concepto censal,Casos,%,Acumulado %,
